In [44]:
# import libraries
import pandas as pd

# load data
profiles = pd.read_csv('../csv/19.01.2026/profiles.csv')
predictions = pd.read_csv('../csv/10.02.2026/predictions_rows.csv')
fixtures = pd.read_csv('../csv/10.02.2026/fixtures_rows(1).csv')

# Merge predictions with profiles on user_id
merged = predictions.merge(profiles, left_on='user_id', right_on='id', how='left', suffixes=('_pred', '_profile'))

# Merge with fixtures on fixture_id
merged = merged.merge(fixtures, left_on='fixture_id', right_on='id', how='left', suffixes=('', '_fixture'))
df_shape = merged[["username", "club", "home_team", "away_team", "home_prediction", "away_prediction", "home_score", "away_score", "game_week_id"]]

# remove inactive players
df_players = df_shape.drop(df_shape[df_shape["username"] == "Martinez"].index)

# remove non premier league weeks
df_game_weeks = df_players.drop(df_players[df_players["game_week_id"].isin(["7d6130ac-980a-4352-8ec0-29a06643525e", "ef99d1e9-75a3-4c2a-9ab3-48604f86b4fa"])].index)

# convert all numbers to integers
cols_to_convert = ["home_score", "away_score", "home_prediction", "away_prediction"]

df_game_weeks[cols_to_convert] = df_game_weeks[cols_to_convert].astype(int)

df_final = df_game_weeks

# check the final dataframe
print (df_final.shape)
print (df_final.head())
print (df_final.dtypes)

(7480, 9)
       username               club        home_team          away_team  \
0   🍺The Barman  Tottenham Hotspur  AFC Bournemouth            Arsenal   
1     Stephen O            Arsenal     Leeds United  Tottenham Hotspur   
2   Jim Shirley           West Ham          Everton          Brentford   
3  Steve arnold              Spurs         West Ham            Chelsea   
4    Mjd-⚒️⚒️⚒️   West Ham  united          Burnley            Arsenal   

   home_prediction  away_prediction  home_score  away_score  \
0                0                2           2           3   
1                1                2           1           2   
2                1                1           2           4   
3                1                3           1           5   
4                0                2           0           2   

                           game_week_id  
0  b8e9c830-4504-4e15-9aba-2cf1e3b2de9d  
1  d77ab4fc-0b5e-46dc-8403-2c02eb802dd6  
2  b8e9c830-4504-4e15-9aba-2cf1e3b2de9d 

In [45]:
# clean up team names

team_fixes = {
    # Brighton
    "Brighton": "Brighton & Hove Albion",
    "Brighton and Hove Albion": "Brighton & Hove Albion",

    # Bournemouth
    "Bournemouth": "AFC Bournemouth",

    # Wolves
    "Wolves": "Wolverhampton Wanderers",

    # Man United
    "Man Utd": "Manchester United",
    "Man U": "Manchester United",
    "Manchester Utd": "Manchester United",
    "Manchester United ": "Manchester United",

    # Man City
    "Man City": "Manchester City",

    # Aston Villa
    "Aston Villa ": "Aston Villa",

    # West Ham
    "West Ham": "West Ham United",
    " West Ham": "West Ham United",

    # Crystal Palace
    " Crystal Palace": "Crystal Palace",

    # Brentford
    " Brentford": "Brentford",

    # Leeds
    "Leeds": "Leeds United",
    " Leeds": "Leeds United",

    # Newcastle
    "Newcastle": "Newcastle United",

    # Nottingham Forest
    "Nottm Forest": "Nottingham Forest",

    # Tottenham
    "Tottenham": "Tottenham Hotspur",
    " Tottenham Hotspur": "Tottenham Hotspur",

    # Chelsea
    " Chelsea": "Chelsea"
}

# apply fixes
df_final["home_team"] = df_final["home_team"].replace(team_fixes)
df_final["away_team"] = df_final["away_team"].replace(team_fixes)

# show unique values
print("Home")
print(df_final["home_team"].nunique())
print(sorted(df_final["home_team"].unique()))

print("\nAway")
print(df_final["away_team"].nunique())
print(sorted(df_final["away_team"].unique()))

Home
20
['AFC Bournemouth', 'Arsenal', 'Aston Villa', 'Brentford', 'Brighton & Hove Albion', 'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leeds United', 'Liverpool', 'Manchester City', 'Manchester United', 'Newcastle United', 'Nottingham Forest', 'Sunderland', 'Tottenham Hotspur', 'West Ham United', 'Wolverhampton Wanderers']

Away
20
['AFC Bournemouth', 'Arsenal', 'Aston Villa', 'Brentford', 'Brighton & Hove Albion', 'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leeds United', 'Liverpool', 'Manchester City', 'Manchester United', 'Newcastle United', 'Nottingham Forest', 'Sunderland', 'Tottenham Hotspur', 'West Ham United', 'Wolverhampton Wanderers']


In [46]:
print(df_final.head())
print("home team counts")
print(df_final["home_team"].value_counts())
print("away team counts")
print(df_final["away_team"].value_counts())
print("game week counts")
print(df_final["game_week_id"].value_counts())
print(df_final["game_week_id"].nunique())

       username               club        home_team          away_team  \
0   🍺The Barman  Tottenham Hotspur  AFC Bournemouth            Arsenal   
1     Stephen O            Arsenal     Leeds United  Tottenham Hotspur   
2   Jim Shirley           West Ham          Everton          Brentford   
3  Steve arnold              Spurs  West Ham United            Chelsea   
4    Mjd-⚒️⚒️⚒️   West Ham  united          Burnley            Arsenal   

   home_prediction  away_prediction  home_score  away_score  \
0                0                2           2           3   
1                1                2           1           2   
2                1                1           2           4   
3                1                3           1           5   
4                0                2           0           2   

                           game_week_id  
0  b8e9c830-4504-4e15-9aba-2cf1e3b2de9d  
1  d77ab4fc-0b5e-46dc-8403-2c02eb802dd6  
2  b8e9c830-4504-4e15-9aba-2cf1e3b2de9d  
3  28c25

In [47]:
# Create predicted league table for The Barman
player_name = '🍺The Barman'
player_data = df_final[df_final['username'] == player_name].copy()

# Get all unique teams
all_teams = pd.concat([player_data['home_team'], player_data['away_team']]).unique()
print(f"Found {len(all_teams)} teams")

# Initialize league table dictionary
league_table = []

for team in all_teams:
    # Get all matches for this team
    home_matches = player_data[player_data['home_team'] == team]
    away_matches = player_data[player_data['away_team'] == team]
    
    # Calculate stats
    wins = 0
    draws = 0
    losses = 0
    goals_for = 0
    goals_against = 0
    
    # Home matches
    for _, match in home_matches.iterrows():
        home_goals = match['home_prediction']
        away_goals = match['away_prediction']
        goals_for += home_goals
        goals_against += away_goals
        
        if home_goals > away_goals:
            wins += 1
        elif home_goals == away_goals:
            draws += 1
        else:
            losses += 1
    
    # Away matches
    for _, match in away_matches.iterrows():
        home_goals = match['home_prediction']
        away_goals = match['away_prediction']
        goals_for += away_goals
        goals_against += home_goals
        
        if away_goals > home_goals:
            wins += 1
        elif away_goals == home_goals:
            draws += 1
        else:
            losses += 1
    
    # Calculate points and goal difference
    points = (wins * 3) + (draws * 1)
    goal_diff = goals_for - goals_against
    
    league_table.append({
        'Team': team,
        'Wins': wins,
        'Draws': draws,
        'Losses': losses,
        'Goals For': goals_for,
        'Goals Against': goals_against,
        'Goal Difference': goal_diff,
        'Points': points
    })

# Create DataFrame and sort
league_df = pd.DataFrame(league_table)
league_df = league_df.sort_values(
    by=['Points', 'Goal Difference', 'Goals For', 'Team'],
    ascending=[False, False, False, True]
).reset_index(drop=True)

# Add position index (1-20)
league_df.index = range(1, len(league_df) + 1)

# Get player's favorite club
fav_club = player_data['club'].iloc[0] if len(player_data) > 0 else ''

print(f"\n{player_name} - {fav_club}")
print("="*80)
print(league_df.to_string())

Found 20 teams

🍺The Barman - Tottenham Hotspur
                       Team  Wins  Draws  Losses  Goals For  Goals Against  Goal Difference  Points
1           Manchester City    20      1       1         55             15               40      61
2                   Arsenal    17      5       0         48             14               34      56
3                 Liverpool    16      5       1         51             22               29      53
4                   Chelsea    16      3       3         41             20               21      51
5         Tottenham Hotspur    13      5       4         40             27               13      44
6          Newcastle United    12      6       4         37             25               12      42
7    Brighton & Hove Albion    10      7       5         34             28                6      37
8         Manchester United     9      6       7         33             28                5      33
9               Aston Villa     9      6       7    

## Create Predicted League Tables for All Players

In [48]:
def create_predicted_league_table(player_name, df_final):
    """Create a predicted league table for a specific player based on their predictions"""
    player_data = df_final[df_final['username'] == player_name].copy()
    
    if len(player_data) == 0:
        print(f"No data found for {player_name}")
        return None
    
    # Get all unique teams
    all_teams = pd.concat([player_data['home_team'], player_data['away_team']]).unique()
    
    # Initialize league table dictionary
    league_table = []
    
    for team in all_teams:
        # Get all matches for this team
        home_matches = player_data[player_data['home_team'] == team]
        away_matches = player_data[player_data['away_team'] == team]
        
        # Calculate stats
        wins = 0
        draws = 0
        losses = 0
        goals_for = 0
        goals_against = 0
        
        # Home matches
        for _, match in home_matches.iterrows():
            home_goals = match['home_prediction']
            away_goals = match['away_prediction']
            goals_for += home_goals
            goals_against += away_goals
            
            if home_goals > away_goals:
                wins += 1
            elif home_goals == away_goals:
                draws += 1
            else:
                losses += 1
        
        # Away matches
        for _, match in away_matches.iterrows():
            home_goals = match['home_prediction']
            away_goals = match['away_prediction']
            goals_for += away_goals
            goals_against += home_goals
            
            if away_goals > home_goals:
                wins += 1
            elif away_goals == home_goals:
                draws += 1
            else:
                losses += 1
        
        # Calculate points and goal difference
        points = (wins * 3) + (draws * 1)
        goal_diff = goals_for - goals_against
        
        league_table.append({
            'Team': team,
            'Wins': wins,
            'Draws': draws,
            'Losses': losses,
            'Goals For': goals_for,
            'Goals Against': goals_against,
            'Goal Difference': goal_diff,
            'Points': points
        })
    
    # Create DataFrame and sort
    league_df = pd.DataFrame(league_table)
    league_df = league_df.sort_values(
        by=['Points', 'Goal Difference', 'Goals For', 'Team'],
        ascending=[False, False, False, True]
    ).reset_index(drop=True)
    
    # Add position index (1-20)
    league_df.index = range(1, len(league_df) + 1)
    
    return league_df

# Test with The Barman
player_name = '🍺The Barman'
player_data = df_final[df_final['username'] == player_name]
fav_club = player_data['club'].iloc[0] if len(player_data) > 0 else ''

table = create_predicted_league_table(player_name, df_final)

print(f"\n{player_name} - {fav_club}")
print("="*100)
print(table.to_string())
print(f"\nTotal predictions used: {len(player_data)}")


🍺The Barman - Tottenham Hotspur
                       Team  Wins  Draws  Losses  Goals For  Goals Against  Goal Difference  Points
1           Manchester City    20      1       1         55             15               40      61
2                   Arsenal    17      5       0         48             14               34      56
3                 Liverpool    16      5       1         51             22               29      53
4                   Chelsea    16      3       3         41             20               21      51
5         Tottenham Hotspur    13      5       4         40             27               13      44
6          Newcastle United    12      6       4         37             25               12      42
7    Brighton & Hove Albion    10      7       5         34             28                6      37
8         Manchester United     9      6       7         33             28                5      33
9               Aston Villa     9      6       7         30        

In [50]:
# Generate tables for all players
all_players = df_final['username'].unique()
print(f"Generating predicted league tables for {len(all_players)} players...\n")

for player in sorted(all_players):
    # Get player's favorite club
    player_data = df_final[df_final['username'] == player]
    fav_club = player_data['club'].iloc[0] if len(player_data) > 0 else ''
    
    # Create table
    table = create_predicted_league_table(player, df_final)
    
    if table is not None:
        print(f"\n{'='*100}")
        print(f"{player} - {fav_club}")
        print('='*100)
        
        # Style the table
        styled_table = table.style.set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#4472C4'), ('color', 'white'), 
                                         ('border', '1px solid black'), ('font-weight', 'bold'), 
                                         ('text-align', 'center')]},
            {'selector': 'td', 'props': [('border', '1px solid black'), ('padding', '8px'), 
                                         ('text-align', 'center'), ('color', '#000000')]},
            {'selector': 'tr:nth-child(even)', 'props': [('background-color', '#F2F2F2')]},
            {'selector': 'tr:nth-child(odd)', 'props': [('background-color', '#FFFFFF')]},
            {'selector': 'tr:first-child', 'props': [('background-color', '#90EE90')]},  # Top team in green
        ])
        
        display(styled_table)
        print(f"\nPredictions used: {len(player_data)} matches")
        print("\n")

print(f"\\n✓ All {len(all_players)} tables generated!")

Generating predicted league tables for 34 players...


Alan Taylor-Reed - Liverpool


,Team,Wins,Draws,Losses,Goals For,Goals Against,Goal Difference,Points
1,Liverpool,20,2,0,68,19,49,62
2,Arsenal,15,3,4,47,18,29,48
3,Manchester City,14,6,2,51,24,27,48
4,Chelsea,10,10,2,39,22,17,40
5,Newcastle United,11,7,4,37,26,11,40
6,Manchester United,11,6,5,41,30,11,39
7,Tottenham Hotspur,8,10,4,32,27,5,34
8,Aston Villa,7,8,7,32,30,2,29
9,Brighton & Hove Albion,8,5,9,23,27,-4,29
10,Leeds United,6,10,6,25,26,-1,28



Predictions used: 220 matches



Andy Page - Ipswich


,Team,Wins,Draws,Losses,Goals For,Goals Against,Goal Difference,Points
1,Arsenal,17,5,0,44,8,36,56
2,Manchester City,14,6,2,39,14,25,48
3,Liverpool,13,7,2,39,13,26,46
4,Chelsea,13,7,2,33,11,22,46
5,Tottenham Hotspur,12,6,4,37,15,22,42
6,Manchester United,12,5,5,25,15,10,41
7,Newcastle United,9,9,4,29,21,8,36
8,Aston Villa,9,9,4,27,19,8,36
9,Brighton & Hove Albion,9,7,6,21,19,2,34
10,Brentford,7,8,7,23,24,-1,29



Predictions used: 220 matches



Bob sullivan  - Chelsea 


...


Predictions used: 220 matches



big k  - west ham 


,Team,Wins,Draws,Losses,Goals For,Goals Against,Goal Difference,Points
1,Arsenal,18,3,1,55,19,36,57
2,Manchester City,14,6,2,59,30,29,48
3,West Ham United,14,1,7,31,23,8,43
4,Manchester United,10,9,3,45,28,17,39
5,Chelsea,10,7,5,44,29,15,37
6,Newcastle United,9,9,4,69,32,37,36
7,Brighton & Hove Albion,8,9,5,29,26,3,33
8,Aston Villa,8,8,6,37,29,8,32
9,Liverpool,8,7,7,45,34,11,31
10,Crystal Palace,8,7,7,34,31,3,31



Predictions used: 220 matches



🍺The Barman - Tottenham Hotspur


,Team,Wins,Draws,Losses,Goals For,Goals Against,Goal Difference,Points
1,Manchester City,20,1,1,55,15,40,61
2,Arsenal,17,5,0,48,14,34,56
3,Liverpool,16,5,1,51,22,29,53
4,Chelsea,16,3,3,41,20,21,51
5,Tottenham Hotspur,13,5,4,40,27,13,44
6,Newcastle United,12,6,4,37,25,12,42
7,Brighton & Hove Albion,10,7,5,34,28,6,37
8,Manchester United,9,6,7,33,28,5,33
9,Aston Villa,9,6,7,30,26,4,33
10,Crystal Palace,6,7,9,22,31,-9,25



Predictions used: 220 matches


\n✓ All 34 tables generated!
